<img src="https://pbs.twimg.com/profile_images/1523691369763684357/FPimkdzj_400x400.jpg" align="right" style="float" width="200">
<font color="#0096d2"><h1 align="left">Datamecum: Programa técnico intensivo en data science.</h1></font>
<font color="#00628a"><h2 align="left">Módulo No supervisado</h2></font> 
<font color="#00628a"><h2 align="left">Anomalías con Regresión/Reconstrucción</h2></font>

# 1) Autoencoder

Generamos un autoencoder y detectamos anomalías en función de su error de reconstrucción. Utilizamos 5000 datos de una serie temporal de latidos del corazón (ECG), cada uno con 140 puntos.
Están etiquetados como 0 (correspondiente a un ritmo anormal) o 1 (correspondiente a un ritmo normal)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

In [ ]:
# Leemos los datos
dataframe = pd.read_csv('./data/ecg.csv', header=None)
raw_data = dataframe.values
dataframe.head()

## Procesamiento de los datos

In [ ]:
# la última columna contiene las etiquetas (0 = ritmo normal, 1 = ritmo anómalo)
labels = 

# nuestros datos son el resto de puntos
data = 

In [ ]:
# normalizamos los datos entre [0,1]

min_val = tf.reduce_min(train_data)
max_val = tf.reduce_max(train_data)

train_data =
test_data =

In [ ]:
# Entrenaremos el codificador automático usando solo los ritmos normales.

# Separar los ritmos normales de los ritmos anormales.

train_labels = train_labels.astype(bool)
test_labels = test_labels.astype(bool)

# filtramos para ritmos nomales 

normal_train_data = 
normal_test_data = 

# los anómalos son los que tienen falso en cada uno (~ es el negado)

anomalous_train_data = 
anomalous_test_data = 

In [ ]:
# pintamos un ECG normal y uno anómalo

plt.grid()
plt.plot(np.arange(140), #COMPLETE)
plt.title("A Normal ECG")
plt.show()

plt.grid()
plt.plot(np.arange(140), #COMPLETE)
plt.title("An Anomalous ECG")
plt.show()

## Carga del modelo

In [ ]:
# definición del modelo (ahora no es necesario entender el modelo!! Cuando veamos deep learning seremos capaces!)

class AnomalyDetector(Model):
    def __init__(self):
        super(AnomalyDetector, self).__init__()
        self.encoder = tf.keras.Sequential([
            layers.Dense(32, activation="relu"),
            layers.Dense(16, activation="relu"),
            layers.Dense(8, activation="relu")
        ])
        self.decoder = tf.keras.Sequential([
            layers.Dense(16, activation="relu"),
            layers.Dense(32, activation="relu"),
            layers.Dense(140, activation="sigmoid")
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# nos quedamos con que tengo un modelo que predice cuando hago uso de él
autoencoder = AnomalyDetector()
autoencoder.compile(optimizer='adam', loss='mae')

In [ ]:
# para que mi modelo aprenda, tenemos que entrenarlo con los datos normales sólo (estamos buscando anomalías!)

history = autoencoder.fit(normal_train_data, normal_train_data, 
          epochs=50, 
          batch_size=512,
          shuffle=True, verbose = 1)

In [ ]:
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()
plt.show()

In [ ]:
# lo pasas a np para poder pintarlo, porque sino son tensores por dentro

# Pasamos por el modelo los ECG normales -> la predicción de mi modelo es decoded_data
encoded_data = autoencoder.encoder(normal_test_data).numpy() # codifica los datos 
decoded_data = autoencoder.decoder(encoded_data).numpy() # decodifica los datos 


## Detección de anomalías

Clasificará un ECG como anómalo si el error de reconstrucción es mayor que una desviación estándar de los ejemplos de entrenamiento normales. Primero, tracemos un ECG normal del conjunto de entrenamiento, la reconstrucción después de que el codificador automático lo codifique y decodifique, y el error de reconstrucción.

In [ ]:
# dibujamos una ECG y el valor predicho por el modelo

# con normal_test_data

plt.plot( # COMPLETE , 'b')
plt.plot(decoded_data[0], 'r')
plt.fill_between(np.arange(140), decoded_data[0], normal_test_data[0], color='lightcoral')
plt.legend(labels=["Input", "Reconstruction", "Error"])
plt.show()

In [ ]:
# probamos con uno anómalo

# LE PASAMOS LOS ANÓMALOS
encoded_data = 
decoded_data = 

# Lo pintamos 

plt.plot( # COMPLETE , 'b')
plt.plot(decoded_data[0], 'r')
plt.fill_between(np.arange(140), decoded_data[0], anomalous_test_data[0], color='lightcoral')
plt.legend(labels=["Input", "Reconstruction", "Error"])
plt.show()

## Error de reconstrucción

In [ ]:
# vemos cuanto vale la desviación entre la línea 'azul' (ECG real) y la predicción de mi algoritmo

np.sum(np.abs( decoded_data[0] - anomalous_test_data[0]))/140.

In [ ]:
# error de reconstrucción 
reconstructions = autoencoder.predict(normal_train_data)
train_loss = tf.keras.losses.mae(reconstructions, normal_train_data)

plt.hist(train_loss[None,:], bins=50)
plt.xlabel("Train loss")
plt.ylabel("No of examples")
plt.show()

## EJERCICIO

Detectar anomalías calculando si el error de reconstrucción es mayor que un umbral fijo. 


In [ ]:
threshold = 

print("Threshold: ", threshold)

In [ ]:
# podríamos ver la distribución de errores de los anómalos
# estos son sólo los datos anómalos porque ya me los he separado antes

reconstructions = 
test_loss = 

plt.hist(test_loss[None, :], bins=50)
plt.xlabel("Test loss")
plt.ylabel("No of examples")
plt.show()